# 物体检测——批量测试图片脚本
## 使用说明
本脚本用于测试ModelArts模型管理中物体检测模型的批量图片测试

测试步骤：

（1）填写 input_dir 参数，该参数指定了输入图片所在的目录，需要您提前上传图片到Notebook的某个路径；

（2）填写 output_dir 参数，该参数指定了输出结果保存目录；

（3）到ModelArts模型管理中查看您要测试的模型，点击查看模型详情，找到model_id和AI引擎，将model_id复制填写到下面的test_model_id参数中，然后点击本页面顶部菜单栏的 “Kernel”-> “Change kernel”，选择与您的模型一致的引擎；


（4）点击本页面顶部菜单栏的 “Cell”-> “Run All”，本脚本将从输入图片目录逐张读取图片，每张图片将得到两个输出文件，一个是模型预测结果json文件，另一个是将预测框画出来的图片文件，所有输出文件将会被保存到输出结果目录。

## 注意事项：
（1）本脚本的当前版本只在 Tensorflow 训练的模型上经过测试；

（2）如果您的模型预测结果标签为中文，则需要自行修改代码，以便正确地在图片上打印中文标签名

In [1]:
# 请填写如下三个参数
input_dir = 'case_dev/mask_detect/datasets/images/yes_mask'  # 输入图片所在的路径
output_dir = 'case_dev/mask_detect/datasets/images/yes_mask_output'  # 输出结果保存路径
test_model_id = 'ed026f9b-7e5f-48a5-880f-250d231617fa'  # 待测试模型的id

In [2]:
# 对路径做检查
import os
ROOT_DIR = '/home/ma-user/work'
input_dir = os.path.join(ROOT_DIR, input_dir)
output_dir = os.path.join(ROOT_DIR, output_dir)
if not os.path.exists(input_dir):
    raise Exception("输入图片目录 input_dir 不存在，请检查修改！")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### 本脚本将使用 ModelArts SDK 来部署测试模型，我们需要对sdk中的原有一个predictor.py进行修改才可以跑通本脚本，为了方便，可以直接使用已经修改好的脚本来覆盖原有的脚本

In [3]:
import shutil
shutil.copy('./predictor.py', '/home/ma-user/modelarts-sdk/modelarts/predictor.py')

'/home/ma-user/modelarts-sdk/modelarts/predictor.py'

In [4]:
# 将模型部署为本地服务
import cv2
import json
import codecs
import datetime
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from modelarts.session import Session
from modelarts.model import Model
from modelarts.config.model_config import ServiceConfig
session = Session()
Model.configure_tf_infer_environ(device_type='GPU')  # 如果不是使用 TF 训练的模型，则屏蔽这一行
model_instance = Model(session, model_id=test_model_id)
configs = [ServiceConfig(model_id=model_instance.model_id, weight="100", specification="local", instance_count=1)]
predictor_instance = model_instance.deploy_predictor(configs=configs)  # 部署为本地服务Predictor

Configuring tensorflow local inference environment ... 
Successfully configure tensorflow local inference environment
Service name is service-0210-053512
model_local_path is /home/ma-user/modelarts/model_location/model_id_ed026f9b-7e5f-48a5-880f-250d231617fa/
Successfully download file obs-duqi/case_dev/mask_detect/model_snapshots/V0008/model from OBS to local /home/ma-user/modelarts/model_location/model_id_ed026f9b-7e5f-48a5-880f-250d231617fa/1/
local_service_port is 127.0.0.1:6060
Deploying the local service ...
Successfully deployed the local service.


In [5]:
# 开始图片测试
now = datetime.datetime.now()
sub_dir = ('%s%02d%02d%02d%02d%02d') % (now.year, now.month, now.day, now.hour, now.minute, now.second)
final_output_dir = os.path.join(output_dir, sub_dir)
if not os.path.exists(final_output_dir):
    os.mkdir(final_output_dir)
files = os.listdir(input_dir)
count = 0
for file_name in files:
    file_path = os.path.join(input_dir, file_name)
    try:
        img = Image.open(file_path)
    except Exception as e:
        print('%s 打开失败，请检查是否是有效图片' % file_name)
        continue
    predict_result = predictor_instance.predict(data=img, data_type='images')
    predict_result = json.loads(predict_result, encoding='utf8')
    classes = predict_result.get('detection_classes', None)
    boxes = predict_result.get('detection_boxes', None)
    scores = predict_result.get('detection_scores', None)

    if classes is not None:
        img_copy = np.array(img.convert('RGB')).copy()
        for i in range(len(classes)):  # 绘制水平框
            box = boxes[i]
            y1, x1, y2, x2 = [int(float(v)) for v in box]
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), thickness=2)
            text = classes[i] + '-%s' % str(float(scores[i]) * 100)[:4] + '%'
            cv2.putText(img_copy, text, (x1, y1 - 3), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)  # 绘制标签名
    cv2.imwrite(os.path.join(final_output_dir, file_name), img_copy[:, :, ::-1])
    with codecs.open(os.path.join(final_output_dir, file_name.rsplit('.', 1)[0] + '.json'), 'w', 'utf-8') as f:
        json.dump(predict_result, f, indent=4, ensure_ascii=False)
    count += 1
    print('%d, %s 预测成功' % (count, file_name))
print('图片测试完成，总共成功测试 %d 张图片' % count)
print('测试结果保存在：%s' % final_output_dir)
os.system("cd %s; zip -r %s.zip %s" % (output_dir, sub_dir, sub_dir))
print('测试结果已打包为：%s.zip，您可以下载查看。' % sub_dir)

1, d046_1580790992772.jpg 预测成功
2, b007_1580797660197.jpg 预测成功
3, b004_1580797660148.jpg 预测成功
4, d048_1580790992792.jpg 预测成功
图片测试完成，总共成功测试 4 张图片
测试结果保存在：/home/ma-user/work/case_dev/mask_detect/datasets/images_output/20200209213523
测试结果已打包为：20200209213523.zip，您可以下载查看。
